In [3]:
# pip install catboost

In [5]:
# pip install xgboost

In [6]:
import cv2
import numpy as np
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import serial

ModuleNotFoundError: No module named 'serial'

In [ ]:


# Load the dataset
X, y = load_data()

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Initialize the CatBoost and XGBoost models
cb_model = CatBoostRegressor(iterations=1000, learning_rate=0.1)
xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.1)

# Train the models on the training data
cb_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

# Initialize the GSM module
ser = serial.Serial('COM3', 9600)

# Initialize the face detector
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Capture video from the camera
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw a rectangle around each face
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Crop the face from the frame
        face = gray[y:y+h, x:x+w]

        # Resize the face to a fixed size
        face = cv2.resize(face, (100, 100))
       
        # Make predictions on the face
        cb_pred = cb_model.predict(face.reshape(1, -1))
        xgb_pred = xgb_model.predict(face.reshape(1, -1))

        # Get the label with the highest prediction
        label = np.argmax([cb_pred, xgb_pred])

        # If the label is unknown, send an alert to the mobile phone
        if label == 'unknown':
            ser.write(b'ALERT: Unknown face detected!')

# Release the camera and close the window
cap.release()
cv2.destroyAllWindows()
